# Guess My Word

#### Authors:
v1.0 (Spring 2020) William Gan, Kannan Ramchandran

v2.0 (Spring 2022) Andy Dong

## Introduction

In Guess My Word http://hryanjones.com/guess-my-word/, your goal is to make guesses at a secret word that the computer has generated. After each guess, the game tells you whether your guess is before or after the secret word lexicographically. Being bad at it, Andy wants to write a program to solve the problem optimally. In this lab, you'll explore a couple ideas he has and help him write the program.

In [2]:
import matplotlib.pyplot as plt
import numpy as np

## Modeling the Game

Suppose we have obtained a list of the words used in the game. Each word has a count that represents relatively how often it appears in the English language. Suppose the secret word is chosen proportionally to this count. The code in the cells below simulates the game. There are a few things to note:

- The count is proportional to the probability that each word is chosen, and will be the probability distribution when normalized (that is, when divided by the sum of all counts).
- In this version of the game, we're assuming guesses are only allowed to come from the list of possible words used.
- We're also assuming that you aren't told the secret word if you guess it correctly. If your guess matches the secret word, the game will tell you that the secret word is lexicographically after your guess. You have to deduce it yourself.

In [3]:
from words import WORD_COUNT

In [4]:
type(WORD_COUNT)

dict

In [5]:
total_count = sum(WORD_COUNT.values())
sorted_words = sorted(list(WORD_COUNT))
sorted_word_counts = np.array([WORD_COUNT[w] for w in sorted_words])
cumulative_counts = np.cumsum(sorted_word_counts)

In [6]:
print(f'Total count: {total_count}')
print('First 10 sorted words:', *sorted_words[:10])
print(f'Counts associated with these words: {list(sorted_word_counts[:10])}')
print(f'Cumulative counts associated with these words: {list(cumulative_counts[:10])}')

Total count: 37009
First 10 sorted words: abstain accept across action active adhere afraid against alive allegory
Counts associated with these words: [2, 24, 111, 122, 23, 1, 116, 221, 16, 1]
Cumulative counts associated with these words: [2, 26, 137, 259, 282, 283, 399, 620, 636, 637]


The following function uses binary search to find the word associated with an index ranging from $0$ to total_count minus $1$. For example, `binary_search_word_generation(126)` will return the third word, "across". Take a minute to convince yourself why before moving on.

In [7]:
def binary_search_word_generation(index):
    assert 0 <= index < total_count
    low, high = 0, len(sorted_words) - 1
    while low < high:
        mid = (low + high) // 2
        if cumulative_counts[mid] > index:
            high = mid
        else:
            low = mid + 1
    return sorted_words[low]

In [8]:
binary_search_word_generation(126)

'across'

**In the following cell, implement `generate_secret_word` using `binary_search_word_generation`.**

In [9]:
def generate_secret_word():
    # BEGIN YOUR SOLUTION
    i = np.random.randint(total_count)
    return binary_search_word_generation(i)
    # END YOUR SOLUTION

This cell creates a game by generating a secret word and returning `guess_function`, which is a function that returns True if the guess is lexicographically "less than" the secret word, and False otherwise.

In [10]:
def create_game(secret_word):
    def guess_function(guess):
        assert guess in WORD_COUNT
        return guess < secret_word
    return guess_function

## Binary Search Idea

The first idea is to ignore the counts and perform binary search on the possible words. In the cell below, `binary_search_game` receives a `guess_function` which was created by a call to `create_game`. Your code should use this `guess_function` to deduce what the secret word is, as well as keep track of how many guesses it took. **In the cell below, write code that returns the deduced secret word and number of guesses required**.  

*Hint: If you want help implementing binary search, you could model it off of `binary_search_word_generation`.*

In [163]:
def binary_search_game(guess_function):
    guesses = 0
    deduced_word = None
    # BEGIN YOUR SOLUTION

    min_ = 0
    max_ = len(sorted_words)-1

    guesses = 0
    while max_-min_ >= 1:
        guesses += 1

        guess = (max_-min_)//2 + min_

        word = sorted_words[guess]

        if guess_function(word):
            min_ = guess+1
        else:
            max_ = guess

    if not guess_function(sorted_words[min_]):
        return sorted_words[min_], guesses

    return sorted_words[min_+1], guesses
    
    # END YOUR SOLUTION
    # return deduced_word, guesses

Use the following function to test your code. **While your implementation may not get the exact same number of guesses, it should take 9 or 10 guesses**. **It should also be deducing the correct word.**

In [164]:
def test_binary_search_game():
    '''Sanity checks for binary_search_game.'''
    words = ['find', 'time', 'great', 'anyone', 'lying']
    return [binary_search_game(create_game(w)) for w in words]

In [165]:
print('Staff solution:', [('find', 10), ('time', 10), ('great', 10), ('anyone', 10), ('lying', 9)])
print('Your solution: ', test_binary_search_game())

Staff solution: [('find', 10), ('time', 10), ('great', 10), ('anyone', 10), ('lying', 9)]
Your solution:  [('find', 10), ('time', 10), ('great', 10), ('anyone', 10), ('lying', 9)]


## Huffman Coding Idea

The second idea takes the counts into consideration and is similar to [Huffman Coding](https://en.wikipedia.org/wiki/Huffman_coding).

It works as follows:

1. Sort all the words in alphabetical order in a list, and generate a wrapper `HuffmanNode` for each word.
2. Go through all consecutive pairs of nodes. For each pair, consider the sum of the two counts. Find the pair whose sum is minimal.
3. Combine those two nodes into a new node, and make their sum the new count. The node's left child will be the first node in the pair and the node's right child will be the second node in the pair.
4. Replace the two node with the new node in the list at the same position.
5. Go back to step 1, and repeat until there's only one node left. Let it be the root of the Huffman tree.

When creating new nodes, we also need to set `rightmost_word`. It'll be useful when we try to guess the secret word, which works as follows.

1. Set the current node to the root.
2. Guess the `rightmost_word` of the current node's left child.
3. If this is before the secret word, then the secret word must be in the right child. Otherwise, it's in the left child. Take a moment to think about why.
4. Go back to step 2. Repeat until we've reached a leaf in this tree, which will contain the secret word.

**Question:** Why is it important that when generating the Huffman tree, we put the new node at the same position as the original nodes, instead of adding it to an arbitrary place in the list?

**Answer:**

In [166]:
class HuffmanNode:
    def __init__(self, count, word=None):
        self.count = count  # The total count of the words in our own subtree.
        self.word = word  # The will be null in non-leaf nodes.
        self.rightmost_word = word  # The rightmost word in our own subtree.
        self.left_subtree = None  # Our children nodes
        self.right_subtree = None  # Our children nodes

**In the following cell, fill code to generate the HuffmanTree.**

In [167]:
def build_huffman_tree():
    sorted_words = sorted(list(WORD_COUNT))
    nodes = [HuffmanNode(WORD_COUNT[w], w) for w in sorted_words]
    # Will need to combine two nodes len(WORD_COUNT) - 1 times.
    for i in range(len(WORD_COUNT) - 1):
        min_node_idx = None
        min_count = None
        # Choose the consecutive pair of nodes with smallest sum of counts to combine
        for j in range(0, len(nodes) - 1):
            count = nodes[j].count + nodes[j+1].count
            if min_count is None or count < min_count:
                min_node_idx = j
                min_count = count
        new_node = HuffmanNode(min_count)
        # Setup the new node and then replace the current two nodes in the list
        # with the new node.
        # BEGIN YOUR SOLUTION

        new_node.left_subtree = nodes[min_node_idx]
        new_node.right_subtree = nodes[min_node_idx+1]
        new_node.rightmost_word = new_node.right_subtree.rightmost_word

        nodes[min_node_idx] = new_node
        nodes.pop(min_node_idx+1)
        
        # END YOUR SOLUTION
    return nodes[0]

**In the cell below, write code that plays a game with the HuffmanTree and returns the deduced word and number of guesses.**


In [168]:
root = build_huffman_tree()

def huffman_game(guess_function):
    guesses = 0
    deduced_word = None
    # BEGIN YOUR SOLUTION
    global root
    tree = root

    while not tree.word:
        guesses += 1

        if guess_function(tree.left_subtree.rightmost_word):
            tree = tree.right_subtree
        else:
            tree = tree.left_subtree

    deduced_word = tree.word
    
    # END YOUR SOLUTION
    return deduced_word, guesses

Use the following function to test your code. **Your implementation should get the same number of guesses as the staff solution, and it should be deducing the correct word.**

In [169]:
def test_huffman_game():
    '''Sanity checks for huffman_game.
    
    The number of guesses may not be less than binary search for particular
    examples.
    '''
    words = ['find', 'time', 'great', 'anyone', 'lying']
    return [huffman_game(create_game(w)) for w in words]

In [170]:
print('Staff solution:', [('find', 8), ('time', 6), ('great', 8), ('anyone', 8), ('lying', 10)])
print('Your solution: ', test_huffman_game())

Staff solution: [('find', 8), ('time', 6), ('great', 8), ('anyone', 8), ('lying', 10)]
Your solution:  [('find', 8), ('time', 6), ('great', 8), ('anyone', 8), ('lying', 10)]


## Analysis

In this section, we'll try to compare the average number of guesses required by each of the two methods and connect it to entropy.

**In the following cell, compute the entropy of the distribution for the secret word.**

*Note: By default, np.log uses the natural log.*

In [171]:
entropy = None
## BEGIN YOUR SOLUTION

entropy = np.sum([(i/total_count) * np.log(total_count/i) for i in sorted_word_counts])/np.log(2)

## END YOUR SOLUTION
print('Entropy of the distribution:', entropy)

Entropy of the distribution: 7.93574630307442


We'll use sampling to estimate the mean number of guesses for each method. In particular, if $X_i$ is the number of guesses required for trial $i$, we can approximate
$$
E[X] \approx \frac{1}{n} \sum_{i=1}^n X_i
$$

In [172]:
SAMPLES = 100000
binary_search = np.zeros(SAMPLES)
huffman = np.zeros(SAMPLES)
for i in range(SAMPLES):
    secret_word = generate_secret_word()
    _, binary_search[i] = binary_search_game(create_game(secret_word))
    _, huffman[i] = huffman_game(create_game(secret_word))

bins = np.arange(4, 15)
plt.hist(binary_search, bins, alpha=0.5, edgecolor='black', align='left', label='Binary Search')
plt.hist(huffman, bins, alpha=0.5, edgecolor='black', align='left', label='Huffman')
plt.xlabel('Number of Guesses')
plt.xticks(np.arange(4, 15))
plt.legend()
plt.title('Guess My Word')
plt.show()

Let's create confidence intervals for $E[X]$. We know that $\frac{1}{n} \sum_{i=1}^n X_i \sim \mathcal{N}\left(E[X], \frac{var(X)}{n}\right)$ by the CLT. **In the following cell, calculate `sigma_n` and `mu`**. We can use the sample mean to estimate $E[X]$ and sample variance to estimate $var(X)$.

*Note that we're not finding the confidence interval for the number of guesses. The `mu` and `sigma_n` we're finding are for $E[X]$ since we only know the empirical mean but not the true mean. We seek to put a bound on the true mean.*

In [175]:
def confidence_interval_95(samples):
    mu, sigma_n = 0, 0
    # BEGIN YOUR SOLUTION

    mu = np.mean(samples)
    sigma_n = np.sqrt(np.var(samples)/len(samples))
    
    # END YOUR SOLUTION
    return f'{mu:.3f} +- {1.96 * sigma_n:.3f}'

In [176]:
print(f'Binary Search: {confidence_interval_95(binary_search)}')
print(f'Huffman: {confidence_interval_95(huffman)}')

Binary Search: 9.683 +- 0.003
Huffman: 8.266 +- 0.013


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=689c629a-8a71-4920-b171-415d33c91843' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>